In [ ]:
# ==============================================
# [♣과제1] 신한은행 완전 자동화 
# ==============================================
import time, os
import pyautogui as pag
import cv2
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager

# ---------- 설정 ----------
IMG_DIR = r"C:\py_temp\shinhan_img"
BANK_URL = "https://www.shinhan.com/index.jsp"
DOWNLOAD_DIR = r"C:\py_temp\downloads"

IMG_LOGIN_BTN1  = os.path.join(IMG_DIR, "login_btn.png")
IMG_LOGIN_BTN2  = os.path.join(IMG_DIR, "login_btn2.png")
IMG_CERT_NAME   = os.path.join(IMG_DIR, "cert_name.png")
IMG_PW_INPUT    = os.path.join(IMG_DIR, "pw_input.png")
IMG_OK_BTN      = os.path.join(IMG_DIR, "ok_btn.png")
IMG_MENU_DEPOSIT = os.path.join(IMG_DIR, "menu_deposit.png")
IMG_FILE_SAVE    = os.path.join(IMG_DIR, "file_save_btn.png")
IMG_ALL_OPTION   = os.path.join(IMG_DIR, "all_option.png")
IMG_EXCEL_BTN    = os.path.join(IMG_DIR, "excel_btn.png")
IMG_PERIOD_3MONTH = os.path.join(IMG_DIR, "period_3month.png")
IMG_SEARCH_BTN    = os.path.join(IMG_DIR, "search_btn.png")

CERT_PASSWORD = "tlsgksdmsgod123"

# ---------- 유틸 함수 ----------
def wait_and_click_advanced(image_path, step_name="", max_attempts=3, wait_after=2):
    """강화된 이미지 클릭 함수"""
    print(f"\n{'='*70}")
    print(f"★ [{step_name}]")
    print(f"   이미지: {os.path.basename(image_path)}")
    print(f"{'='*70}")
    
    attempts = [
        {'confidence': 0.85, 'grayscale': True,  'timeout': 8},
        {'confidence': 0.75, 'grayscale': True,  'timeout': 8},
        {'confidence': 0.70, 'grayscale': False, 'timeout': 8},
        {'confidence': 0.65, 'grayscale': True,  'timeout': 10},
    ]
    
    for idx, config in enumerate(attempts[:max_attempts], 1):
        print(f"\n[시도 {idx}/{max_attempts}] confidence={config['confidence']}")
        print(f"   찾는 중...", end='', flush=True)
        
        start = time.time()
        dots = 0
        
        while time.time() - start < config['timeout']:
            try:
                pos = pag.locateCenterOnScreen(
                    image_path,
                    confidence=config['confidence'],
                    grayscale=config['grayscale']
                )
            except Exception:
                pos = None
            
            if pos:
                print(f"\n   ✅ 발견! 위치: ({pos.x}, {pos.y})")
                pag.moveTo(pos.x, pos.y, duration=0.3)
                time.sleep(0.3)
                pag.click(pos.x, pos.y)
                print(f"   ✅ 클릭 완료!")
                time.sleep(wait_after)
                return True
            
            dots = (dots + 1) % 4
            print('.' * dots + ' ' * (3 - dots), end='\r', flush=True)
            time.sleep(0.5)
        
        print(f"\n 실패")
    
    print(f"\n 모든 시도 실패!")
    return False

def scroll_down(amount=-500, times=1, wait=1):
    """
     스크롤 내리기 함수
    - amount: 스크롤 양 (음수=아래, 양수=위)
    - times: 반복 횟수
    - wait: 대기 시간
    """
    print(f"\n 페이지 스크롤 내리는 중...", end='', flush=True)
    for i in range(times):
        pag.scroll(amount)
        time.sleep(wait / times)
        print('.', end='', flush=True)
    print(" ✅")

def type_password_slow(password):
    """비밀번호 천천히 입력"""
    print(f"\n{'='*60}")
    print(f"★ [비밀번호 입력]")
    for i, char in enumerate(password, 1):
        pag.write(char)
        time.sleep(0.15)
        if i % 3 == 0:
            print('.', end='', flush=True)
    print("\n 입력 완료!")

def wait_seconds(seconds, message=""):
    """진행 표시가 있는 대기"""
    if message:
        print(f"\n⏳ {message} ({seconds}초)...", end='', flush=True)
    for i in range(seconds):
        time.sleep(1)
        print('.', end='', flush=True)
    print(" ✅")

# ---------- 메인 실행 ----------
print("\n" + "="*80)
print("신한은행 완전 자동화 시스템 (스크롤 추가)")
print("="*80)

input("\n계속하려면 Enter를 누르세요...")

# ==================== STEP 1: 브라우저 시작 ====================
print("\n\n" + "="*80)
print("🌐 STEP 1: 브라우저 시작")
print("="*80)

os.makedirs(DOWNLOAD_DIR, exist_ok=True)

opts = Options()
opts.add_argument("--start-maximized")
prefs = {
    "download.default_directory": DOWNLOAD_DIR,
    "download.prompt_for_download": False,
}
opts.add_experimental_option("prefs", prefs)

driver = webdriver.Chrome(
    service=Service(ChromeDriverManager().install()),
    options=opts
)

driver.get(BANK_URL)
print(f" 접속: {BANK_URL}")
wait_seconds(5, "페이지 로딩 대기")

# ==================== STEP 2: 로그인 ====================
print("\n\n" + "="*80)
print("STEP 2: 로그인 프로세스")
print("="*80)

# 2-1) 메인 로그인
success = wait_and_click_advanced(IMG_LOGIN_BTN1, "메인 로그인", max_attempts=3)
if not success:
    input("실패! 수동으로 클릭 후 Enter...")
wait_seconds(4, "로그인 페이지 로딩")

# 2-2) 공동인증서 로그인
success = wait_and_click_advanced(IMG_LOGIN_BTN2, "공동인증서 로그인", max_attempts=4)
if not success:
    input("실패! 수동으로 클릭 후 Enter...")
wait_seconds(4, "인증서 창 로딩")

# 2-3) 인증서 선택
success = wait_and_click_advanced(IMG_CERT_NAME, "인증서 선택", max_attempts=4)
if not success:
    input("실패! 수동으로 선택 후 Enter...")
time.sleep(2)

# 2-4) 비밀번호 입력창
success = wait_and_click_advanced(IMG_PW_INPUT, "비밀번호 입력창", max_attempts=3)
if not success:
    input("실패! 수동으로 클릭 후 Enter...")
time.sleep(1)

# 2-5) 비밀번호 입력
type_password_slow(CERT_PASSWORD)
time.sleep(1)

# 2-6) 확인 버튼
success = wait_and_click_advanced(IMG_OK_BTN, "확인 버튼", max_attempts=3)
if not success:
    input("실패! 수동으로 클릭 후 Enter...")

print("\n 로그인 완료!")
wait_seconds(5, "메인 페이지 로딩")

# ==================== STEP 3: 조회 메뉴 클릭 ====================
print("\n\n" + "="*80)
print("STEP 3: 조회 메뉴 진입")
print("="*80)


success = wait_and_click_advanced(
    IMG_MENU_DEPOSIT, 
    "입출금/거래내역 조회 메뉴", 
    max_attempts=4,
    wait_after=3
)

if not success:
    input("\n수동으로 조회 메뉴를 클릭한 후 Enter...")

wait_seconds(3, "조회 페이지 로딩")


# ==================== STEP 4: 기간 선택 ====================
print("\n\n" + "="*80)
print("STEP 4: 조회 기간 선택 (3개월)")
print("="*80)

# 3개월 버튼 클릭
print("\n[1단계] 3개월 버튼 클릭")
success = wait_and_click_advanced(IMG_PERIOD_3MONTH, "3개월 버튼", 4, 1)
if not success:
    print("\n 3개월 버튼을 찾을 수 없습니다!")
    input("수동으로 클릭 후 Enter...")

time.sleep(1)

# 조회 버튼 클릭
print("\n[2단계] 조회 버튼 클릭")
success = wait_and_click_advanced(IMG_SEARCH_BTN, "조회 버튼", 4, 3)
if not success:
    print("\n 조회 버튼을 찾을 수 없습니다!")
    input("수동으로 클릭 후 Enter...")

print("\n✅ 기간 선택 및 조회 완료!")
wait_seconds(3, "조회 결과 로딩")

# ★ 핵심: 스크롤 내리기 ★
scroll_down(amount=-500, times=1, wait=2)  # 500픽셀 아래로 스크롤
# 또는 아래 방법 중 선택:
# scroll_down(amount=-300, times=3, wait=1.5)  # 3번 나눠서 스크롤
# pag.press('pagedown')  # Page Down 키 사용

# ==================== STEP 4: 파일저장 클릭 ====================
print("\n\n" + "="*80)
print("STEP 4: 파일저장 버튼 클릭")
print("="*80)

success = wait_and_click_advanced(
    IMG_FILE_SAVE, 
    "파일저장 버튼", 
    max_attempts=4,
    wait_after=2
)

if not success:
    print("\n 파일저장 버튼을 찾을 수 없습니다!")
    input("수동으로 클릭 후 Enter...")

wait_seconds(2, "옵션 창 로딩")

# ==================== STEP 5: 전체 옵션 선택 ====================
print("\n\n" + "="*80)
print("STEP 5: 전체 옵션 선택")
print("="*80)

success = wait_and_click_advanced(
    IMG_ALL_OPTION, 
    "전체 옵션 체크박스", 
    max_attempts=4,
    wait_after=1
)

if not success:
    input("수동으로 클릭 후 Enter...")

time.sleep(1)

# ==================== STEP 6: 엑셀 저장 ====================
print("\n\n" + "="*80)
print("STEP 6: 엑셀 저장")
print("="*80)

success = wait_and_click_advanced(
    IMG_EXCEL_BTN, 
    "엑셀저장 버튼", 
    max_attempts=4,
    wait_after=2
)

if not success:
    input("수동으로 클릭 후 Enter...")

wait_seconds(3, "파일 다운로드 중")

# ==================== 완료 ====================
print("\n\n" + "="*80)
print("🎉모든 작업 완료!🎉")
print("="*80)

print("\n✅ 완료된 작업:")
print("  1. ✅ 로그인")
print("  2. ✅ 조회 메뉴")
print("  3. ✅ 스크롤 다운")  # ★ 추가됨
print("  4. ✅ 파일저장")
print("  5. ✅ 전체 옵션")
print("  6. ✅ 엑셀 저장")

print(f"\n 저장 위치: {DOWNLOAD_DIR}")

try:
    os.startfile(DOWNLOAD_DIR)
    print(f"다운로드 폴더 열림!")
except:
    pass

print("\n" + "="*80)
input("\nEnter를 눌러 종료...")
driver.quit()


신한은행 완전 자동화 시스템 (스크롤 추가)



계속하려면 Enter를 누르세요... 




🌐 STEP 1: 브라우저 시작
 접속: https://www.shinhan.com/index.jsp

..... ✅딩 대기 (5초)...


STEP 2: 로그인 프로세스

★ [메인 로그인]
   이미지: login_btn.png

[시도 1/3] confidence=0.85
.. 찾는 중...
 실패

[시도 2/3] confidence=0.75
.. 찾는 중...
   ✅ 발견! 위치: (644, 216)
   ✅ 클릭 완료!

.... ✅페이지 로딩 (4초)...

★ [공동인증서 로그인]
   이미지: login_btn2.png

[시도 1/4] confidence=0.85
   찾는 중...
   ✅ 발견! 위치: (625, 715)
   ✅ 클릭 완료!

.... ✅창 로딩 (4초)...

★ [인증서 선택]
   이미지: cert_name.png

[시도 1/4] confidence=0.85
   찾는 중...
   ✅ 발견! 위치: (957, 660)
   ✅ 클릭 완료!

★ [비밀번호 입력창]
   이미지: pw_input.png

[시도 1/3] confidence=0.85
   찾는 중...
   ✅ 발견! 위치: (906, 873)
   ✅ 클릭 완료!

★ [비밀번호 입력]
...
 입력 완료!

★ [확인 버튼]
   이미지: ok_btn.png

[시도 1/3] confidence=0.85
   찾는 중...
   ✅ 발견! 위치: (895, 959)
   ✅ 클릭 완료!

 로그인 완료!

..... ✅지 로딩 (5초)...


STEP 3: 조회 메뉴 진입

★ [입출금/거래내역 조회 메뉴]
   이미지: menu_deposit.png

[시도 1/4] confidence=0.85
   찾는 중...
   ✅ 발견! 위치: (1339, 769)
   ✅ 클릭 완료!

... ✅페이지 로딩 (3초)...


STEP 4: 조회 기간 선택 (3개월)

[1단계] 3개월 버튼 클릭

★ [3개월 버튼]
   이미지: perio